In [1]:
from copy import deepcopy

import pandas as pd

from BitBoard import osero
from osero_learn import learn

PLAY_WAY = deepcopy(osero.PLAY_WAY)
del PLAY_WAY["human"]
PLAY_WAY = PLAY_WAY.values()

In [2]:
eva = [[
     1.0, -0.6,  0.6,  0.4,  0.4,  0.6, -0.6,  1.0,
    -0.6, -0.8,  0.0,  0.0,  0.0,  0.0, -0.8, -0.6,
     0.6,  0.0,  0.8,  0.6,  0.6,  0.8,  0.0,  0.6,
     0.4,  0.0,  0.6,  0.0,  0.0,  0.6,  0.0,  0.4,
     0.4,  0.0,  0.6,  0.0,  0.0,  0.6,  0.0,  0.4,
     0.6,  0.0,  0.8,  0.6,  0.6,  0.8,  0.0,  0.6,
    -0.6, -0.8,  0.0,  0.0,  0.0,  0.0, -0.8, -0.6,
     1.0, -0.6,  0.6,  0.4,  0.4,  0.6, -0.6,  1.0
] for i in range(2)]

In [3]:
df = pd.DataFrame({})

check_point = [i for i in range(5, 64, 5)]

for i in range(10):
    print("\r[" + "#" * (i+1) + " " * (10-i+1) + "]", end="")
    for black in PLAY_WAY:
        for white in PLAY_WAY:
            run = learn(\
                black,
                white,
                check_point=check_point,
                seed_num=i,
                eva=eva
            )
            data = run.play()
            df = df.append(data, ignore_index=True)

print("\r[" + "#" * 10 + "]")

[##########]


In [4]:
import matplotlib.pyplot as plt
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.linear_model import Ridge

x_data = df.drop(["turn", "last_score"], axis=1)
y_data = df[["turn_num", "last_score"]]

turn_vari = df["turn_num"].unique()
drop_vari = ["custom_score and score", "custom_score", "score", "None"]

In [5]:
train_score = []
test_score = []

train_MAE = []
test_MAE = []

for turn_num in turn_vari:
    train_score.append([])
    test_score.append([])

    train_MAE.append([])
    test_MAE.append([])

    for drop_list in [["custom_score", "score"], ["custom_score"], ["score"], []]:
        drop_list.append("turn_num")
        x_train, x_test, y_train, y_test = train_test_split(\
            x_data.query("turn_num==%d" % turn_num).drop(drop_list, axis=1),
            y_data.query("turn_num==%d" % turn_num).drop("turn_num", axis=1),
            test_size=0.3,
            random_state=0
        )

        model = Ridge(random_state=0)
        model.fit(x_train, y_train)
        
        train_score[-1].append(model.score(x_train, y_train))
        test_score[-1].append(model.score(x_test, y_test))

        train_predict = model.predict(x_train)
        train_MAE[-1].append(mean_absolute_error(train_predict, y_train))
        test_predict = model.predict(x_test)
        test_MAE[-1].append(mean_absolute_error(test_predict, y_test))

In [6]:
width = 0.3
x_axis = np.array([i + 1 for i in range(len(drop_vari))])

for i in range(len(turn_vari)):
    fig_name = "score of each dropping (number of turn is %d)" % turn_vari[i]
    fig = plt.figure(figsize=(10, 10))
    plt.bar(x_axis, train_score[i], label="train score", width=width)
    plt.bar(x_axis + width, test_score[i], label="test score", width=width)
    plt.xticks(x_axis + width/2, labels=drop_vari, rotation=15)
    plt.legend()
    plt.title(fig_name)
    plt.xlabel("dropped column")
    plt.ylabel("score")
    plt.savefig("fig/" + fig_name)
    # plt.show()
    plt.clf()
    plt.close()

    fig_name = "MAE of each dropping (number of turn is %d)" % turn_vari[i]
    fig = plt.figure(figsize=(10, 10))
    plt.bar(x_axis, train_MAE[i], label="train MAE", width=width)
    plt.bar(x_axis + width, test_MAE[i], label="test MAE", width=width)
    plt.xticks(x_axis + width/2, labels=drop_vari, rotation=15)
    plt.legend()
    plt.title(fig_name)
    plt.xlabel("dropped column")
    plt.ylabel("mean absolute error")
    plt.savefig("fig/" + fig_name)
    # plt.show()
    plt.clf()
    plt.close()

In [7]:
x_axis = np.array([i + 1 for i in range(len(turn_vari))])
x_axis_name = [str(i) for i in turn_vari]

train_score_T = np.array(train_score).T
test_score_T = np.array(test_score).T

train_MAE_T = np.array(train_MAE).T
test_MAE_T = np.array(test_MAE).T

for i in range(len(drop_vari)):
    fig_name = "score of each turn number (dropped column is %s)" % drop_vari[i]
    fig = plt.figure(figsize=(10, 10))
    plt.plot(x_axis_name, train_score_T[i], label="train score")
    plt.plot(x_axis_name, test_score_T[i], label="test score")
    plt.legend()
    plt.title(fig_name)
    plt.xlabel("turn number")
    plt.ylabel("score")
    plt.savefig("fig/" + fig_name)
    # plt.show()
    plt.clf()
    plt.close()

    fig_name = "MAE of each turn number (dropped column is %s)" % drop_vari[i]
    fig = plt.figure(figsize=(10, 10))
    plt.plot(x_axis_name, train_MAE_T[i], label="train MAE")
    plt.plot(x_axis_name, test_MAE_T[i], label="test MAE")
    plt.legend()
    plt.title(fig_name)
    plt.xlabel("turn number")
    plt.ylabel("mean absolute error")
    plt.savefig("fig/" + fig_name)
    # plt.show()
    plt.clf()
    plt.close()